In [ ]:
import pandas as pd
import pdfplumber as pr
import re
import os
import numpy as np
import warnings

In [ ]:
warnings.filterwarnings('ignore')

### 讀取PDF files

In [ ]:
PDF_files = []
input_path = r'../input'
for x in os.listdir(input_path):
    if '.pdf' in x:
        PDF_files.append(x)
#這個檔案不要刪除，很重要
GFI = pd.read_excel(r'../input\202111 政府基金運用資訊 v2.0.xlsx',sheet_name = 'Onshore (Raw data)',skiprows=1)

In [ ]:
#目標要找的圖表
#將有Relative Return 的基金名稱加上(相對報酬)
kind_convert = {
    'Absolute Return':'',
    'Relative Return':'(相對報酬)'
}

target_fund_lists = []
target_name_tables = GFI['Mandate Type']
GFI['Asset Class_CHI'] = GFI['Asset Class'].map(kind_convert)
GFI['final_name'] = GFI['Mandate Type'] + GFI['Asset Class_CHI']
target_fund_lists = list(GFI['final_name'].drop_duplicates().values)


#為了避免LPF及LRF有跟其他基金一樣的名稱，導致結果錯誤，這邊把他們隔離出來
target_fund_lists_勞退 = []
GFI_勞退 = GFI[(GFI['Fund']=='LPF') | (GFI['Fund']=='LRF')]
target_name_tables = GFI_勞退['Mandate Type']
GFI_勞退['Asset Class_CHI'] = GFI_勞退['Asset Class'].map(kind_convert)
GFI_勞退['final_name'] = GFI_勞退['Mandate Type'] + GFI_勞退['Asset Class_CHI']
target_fund_lists_勞退 = list(GFI_勞退['final_name'].drop_duplicates().values)

In [ ]:
#for 退撫
PSPF = pd.DataFrame({})
for PDF_file in PDF_files:
    if '基金運用情形' in PDF_file:
        #11月基金運用情形
        temp_PDF_file = pr.open(input_path+'/'+PDF_file)
        #是一個list
        ps            = temp_PDF_file.pages
        #用range配len跑for迴圈
        for p in range(len(ps)):
            pg = ps[p]
            page_tables = pg.extract_tables()
            try:
                #print(len(page_tables))
                for table_num in range(len(page_tables)):
                    #print(table_num)
                    table_委託經營  = pd.DataFrame(page_tables[table_num])
                    multi          = [table_委託經營.iloc[0,:].dropna(axis=0),table_委託經營.iloc[1,:].fillna('')+table_委託經營.iloc[2,:].fillna('')]
                    mindex         = pd.MultiIndex.from_product(multi,names=['Level1','Level2'])
                    table_最終結果  = pd.DataFrame(columns = mindex, index = table_委託經營.iloc[3:,:].index)
                    table_最終結果[table_委託經營.iloc[0,:].dropna(axis=0)[0]] = table_委託經營.iloc[3:,:].values

                    #將資料補齊
                    #找到基金的日期
                    Funds_Date     = table_最終結果.columns.get_level_values(0)[0]
                    table_最終結果  = table_最終結果[table_最終結果[Funds_Date,'受託機構名稱'] != '合計']
                    table_最終結果[Funds_Date,'Fund'] = 'PSPF'
                    table_最終結果[Funds_Date,'Mandate Type'] = Funds_Date.split('（')[0]
                    #將報酬率補起來，有的是目標，有的是指標
                    try:
                        table_最終結果[Funds_Date,'目標報酬率\n(%)'] = table_最終結果[Funds_Date,'目標報酬率\n(%)'].fillna(method = 'ffill')
                    except:
                        table_最終結果[Funds_Date,'指標報酬率\n(%)'] = table_最終結果[Funds_Date,'指標報酬率\n(%)'].fillna(method = 'ffill')
                    table_最終結果[Funds_Date,'受託機構名稱'] = table_最終結果[Funds_Date,'受託機構名稱'].str.replace('投信','')
                    #補開始及結束的日期
                    table_最終結果[Funds_Date,'From']  = (str(int(re.findall(r'\d+年\d+月\d+日',Funds_Date)[0][:3])+1911)+str(re.findall(r'\d+年\d+月\d+日',Funds_Date)[0][3:])).replace('日','')
                    table_最終結果[Funds_Date,'As of'] = (str(int(re.findall(r'\d+年\d+月\d+日',Funds_Date)[1][:3])+1911)+str(re.findall(r'\d+年\d+月\d+日',Funds_Date)[1][3:])).replace('日','')
                    for target in ['年','月']:
                        table_最終結果[Funds_Date,'From']  = table_最終結果[Funds_Date,'From'].str.replace(target,'/')
                        table_最終結果[Funds_Date,'As of'] = table_最終結果[Funds_Date,'As of'].str.replace(target,'/')
                    PSPF = PSPF.append(pd.DataFrame(table_最終結果.values))
            except:
                print(str(p)+'有問題or 找不到目標圖表')
            finally:
                pass
        PSPF.columns = ['SITE','Mandate Size','AUM','YTD %','Since Inception %','Ranking','Target','Fund','Mandate Type','From','As of']

In [ ]:
#for 勞保
for PDF_file in PDF_files:
    if '勞工保險基金' in PDF_file:
        PDF_file = pr.open(input_path+'/'+PDF_file)

        有勞保目標基金的頁數 = []
        勞保目標基金的名稱   = []
        #是一個list
        ps              = PDF_file.pages
        #從0開始
        #用range配len跑for迴圈
        for p in range(len(ps)):
            table_num = 0
            pg = ps[p]
            page_text = pg.extract_text().replace(' ','')
            page_text = page_text.replace('第1次','年第一次委託經營')
            for target_fund in target_fund_lists:
                #如果我的目標基金有在TEXT內
                if str(target_fund)!= 'nan':
                    if target_fund.replace(' ','') in page_text:
                        有勞保目標基金的頁數.append((p,table_num))
                        勞保目標基金的名稱.append(target_fund)
                        table_num+=1

        print(有勞保目標基金的頁數)
        print(勞保目標基金的名稱)
        LIF = pd.DataFrame({})
        PDF_file = pr.open(input_path+'/'+PDF_files[1])
        勞保目標業及表名 = list(zip(有勞保目標基金的頁數,勞保目標基金的名稱))
        for page_table, table_name in 勞保目標業及表名:
            ps = PDF_file.pages
            pg = ps[page_table[0]]
            page_table = pg.extract_tables()[page_table[1]]
            table_勞保 = pd.DataFrame(page_table)
            #處理表格內的資料
            table_勞保.iloc[0] = table_勞保.iloc[0].fillna('',axis=0)
            table_勞保.iloc[1] = table_勞保.iloc[1].fillna('',axis=0)
            Date_Start = re.search(r'(\d+.\d+.\d+)~(\d+.\d+.\d+)',table_勞保.iloc[0,4]).group(1)
            Date_End   = re.search(r'(\d+.\d+.\d+)~(\d+.\d+.\d+)',table_勞保.iloc[0,4]).group(2)
            table_勞保.iloc[0,4] = re.sub(r'(\d+.\d+.\d+)~(\d+.\d+.\d+)','',table_勞保.iloc[0,4])
            table_勞保_final           = table_勞保.iloc[2:,:]
            table_勞保_final.columns   = ['SITE','Mandate Size','AUM','YTD %','Since Inception %','Ranking','Target']
            table_勞保_final['Target'] = table_勞保_final['Target'].fillna(method = 'ffill')
            table_勞保_final[['Fund','Mandate Type','From','As of']]=['LIF',table_name,Date_Start,Date_End]
            table_勞保_final['From']   = (str(int(re.findall(r'\d+.\d+.\d+',Date_Start)[0][:3])+1911)+str(re.findall(r'\d+.\d+.\d+',Date_Start)[0][3:])).replace('.','/')
            table_勞保_final['As of']  = (str(int(re.findall(r'\d+.\d+.\d+',Date_End)[0][:3])+1911)+str(re.findall(r'\d+.\d+.\d+',Date_End)[0][3:])).replace('.','/')
            table_勞保_final = table_勞保_final[table_勞保_final['SITE']!='合計']

            LIF = LIF.append(table_勞保_final)

In [ ]:
#for 國民年金
for PDF_file in PDF_files:
    if '國民年金保險基金' in PDF_file:
        PDF_file = pr.open(input_path+'/'+PDF_file)

        有國民年金目標基金的頁數 = []
        國民年金目標基金的名稱   = []
        #從0開始
        #是一個list
        ps              = PDF_file.pages
        #用range配len跑for迴圈
        for p in range(len(ps)):
            table_num = 0
            pg = ps[p]
            page_text = pg.extract_text().replace(' ','')
            page_text = page_text.replace('102年度續約1委託帳戶','國民年金保險基金102 年度委託帳戶續約1')

            for target_fund in target_fund_lists:
                #如果我的目標基金有在TEXT內
                if str(target_fund)!= 'nan':
                    if target_fund in page_text:
                        有國民年金目標基金的頁數.append((p,table_num))
                        國民年金目標基金的名稱.append(target_fund)
                        table_num+=1

        print(有國民年金目標基金的頁數)
        print(國民年金目標基金的名稱)

        NPIF = pd.DataFrame({})
        PDF_file = pr.open(input_path+'/'+PDF_files[3])
        國民年金目標業及表名 = list(zip(有國民年金目標基金的頁數,國民年金目標基金的名稱))
        for page_table, table_name in 國民年金目標業及表名:
            ps = PDF_file.pages
            pg = ps[page_table[0]]
            page_table = pg.extract_tables()[page_table[1]]
            table_國民年金 = pd.DataFrame(page_table)
            #處理表格內的資料
            table_國民年金.iloc[0] = table_國民年金.iloc[0].fillna('',axis=0)
            table_國民年金.iloc[1] = table_國民年金.iloc[1].fillna('',axis=0)
            Date_Start = re.search(r'(\d+.\d+.\d+)~(\d+.\d+.\d+)',table_國民年金.iloc[0,4]).group(1)
            Date_End = re.search(r'(\d+.\d+.\d+)~(\d+.\d+.\d+)',table_國民年金.iloc[0,4]).group(2)
            table_國民年金.iloc[0,4] = re.sub(r'(\d+.\d+.\d+)~(\d+.\d+.\d+)','',table_國民年金.iloc[0,4])
            table_國民年金_final = table_國民年金.iloc[2:,:]
            table_國民年金_final.columns = ['SITE','Mandate Size','AUM','YTD %','Since Inception %','Ranking','Target']
            table_國民年金_final['Target'] = table_國民年金_final['Target'].fillna(method = 'ffill')
            table_國民年金_final[['Fund','Mandate Type','From','As of']]=['NPIF',table_name,Date_Start,Date_End]
            table_國民年金_final['From'] = (str(int(re.findall(r'\d+.\d+.\d+',Date_Start)[0][:3])+1911)+str(re.findall(r'\d+.\d+.\d+',Date_Start)[0][3:])).replace('.','/')
            table_國民年金_final['As of'] = (str(int(re.findall(r'\d+.\d+.\d+',Date_End)[0][:3])+1911)+str(re.findall(r'\d+.\d+.\d+',Date_End)[0][3:])).replace('.','/')
            table_國民年金_final = table_國民年金_final[table_國民年金_final['SITE']!='合計']

            NPIF = NPIF.append(table_國民年金_final)

In [ ]:
target_fund_lists.append('110 年第一次委託經營')
target_fund_lists

In [ ]:
#for 勞退
#問題小大，名稱需要統一一下會比較簡單
for PDF_file in PDF_files:
    if '勞工退休基金' in PDF_file:
        PDF_file = pr.open(input_path+'/'+PDF_file)
        #從0開始
        有勞退目標基金的頁數 = []
        勞退目標基金的名稱   = []
        #是一個list
        ps              = PDF_file.pages
        #用range配len跑for迴圈
        for p in range(len(ps)):
            table_num = 0
            pg = ps[p]
            page_text = pg.extract_text()
            #新制
            if p <= 11:
                if '96 年第一次委託經營(續約 4)' in page_text:
                    page_text = page_text.replace('96 年第一次委託經營(續約 4)','96年第一次委託經營（續約4）')
                if '97 年第一次委託經營(續約 4)' in page_text:
                    page_text = page_text.replace('97 年第一次委託經營(續約 4)','97年第一次委託經營（續約4）')
                if '98 年第一次委託經營(續約 4)' in page_text:
                    page_text = page_text.replace('98 年第一次委託經營(續約 4)','98 年第一次委託經營（續約4）')
                if '99 年第二次委託經營(續約 2)' in page_text:
                    page_text = page_text.replace('99 年第二次委託經營(續約 2)','99 年第二次委託經營（續約2） ')
                if '100 年第一次委託經營(續約 2)(相對報酬)' in page_text:
                    page_text = page_text.replace('100 年第一次委託經營(續約 2)(相對報酬)','100 年第一次委託經營( 續約2 )(相對報酬)')
                if '101 年第一次委託經營(續約 2)(相對報酬)' in page_text:
                    page_text = page_text.replace('101 年第一次委託經營(續約 2)(相對報酬)','101 年第一次委託經營( 續約 2)(相對報酬)')
                if '101 年第二次委託經營(續約 2)' in page_text:
                    page_text = page_text.replace('101 年第二次委託經營(續約 2)','101 年第二次委託經營( 續約 2)')
                if '102 年第二次委託經營(續約) (相對報酬)' in page_text:
                    page_text = page_text.replace('102 年第二次委託經營(續約) (相對報酬)','102 年第二次委託經營 ( 續約 )(相對報酬)')
                if '103 年第一次委託經營第二期(續約)' in page_text:
                    page_text = page_text.replace('103 年第一次委託經營第二期(續約)','103 年第一次委託經營第二期(續約)')
                if '103 年第二次委託經營(續約)(相對報酬)' in page_text:
                    page_text = page_text.replace('103 年第二次委託經營(續約)(相對報酬)','103 年第二次委託經營(相對報酬)')
                if '104 年第一次委託經營  (續約)' in page_text:
                    page_text = page_text.replace('104 年第一次委託經營  (續約)','104 年第一次委託經營第一期( 續約 )')
                if '106 年第一次委託經營' in page_text:
                    page_text = page_text.replace('106 年第一次委託經營','106 年第一次委託經營')
                if '107 年第一次委託經營(相對報酬)' in page_text:
                    page_text = page_text.replace('107 年第一次委託經營(相對報酬)','107 年第一次委託經營(相對報酬)')
                if '107 年第二次委託經營' in page_text:
                    page_text = page_text.replace('107 年第二次委託經營','107 年第二次委託經營')
                if '109 年第一次委託經營' in page_text:
                    page_text = page_text.replace('109 年第一次委託經營','109 年第一次委託經營')
                #######
                if '110 年第一次委託經營' in page_text:
                    page_text = page_text.replace('110 年第一次委託經營','110 年第一次委託經營')
            else:
                #舊制
                if '國內 98 年第二次委託經營(續約 3)' in page_text:
                    page_text = page_text.replace('國內 98 年第二次委託經營(續約 3)','國內 98 年第二次委託經營( 續約 3)')
                if '國內 101 年第一次委託經營(相對報酬)(續約 2)' in page_text:
                    page_text = page_text.replace('國內 101 年第一次委託經營(相對報酬)(續約 2)','國內 101 年第一次委託經營( 續約2 )(相對報酬)')
                if '國內 101 年第二次委託經營(續約 2)' in page_text:
                    page_text = page_text.replace('國內 101 年第二次委託經營(續約 2)','國內 101 年第二次委託經營( 續約2 )')
                if '國內 104 年第一次委託經營(續約)' in page_text:
                    page_text = page_text.replace('國內 104 年第一次委託經營(續約)','國內104 年第一次委託經營第一期( 續約 )')
                if '國內 106 年第一次委託經營' in page_text:
                    page_text = page_text.replace('國內 106 年第一次委託經營','國內 106 年第一次委託經營第一期')

            


            for target_fund in target_fund_lists_勞退:
                #如果我的目標基金有在TEXT內
                if str(target_fund)!= 'nan':
                    if target_fund in page_text:
                        有勞退目標基金的頁數.append((p,table_num))
                        勞退目標基金的名稱.append(target_fund)
                        table_num+=1
                       
    
        有勞退目標基金的頁數_去重複 = list(set([x[0] for x in 有勞退目標基金的頁數]))

        #這邊之後要改
        ###
        勞退目標基金的名稱.insert(15,'110年第一次委託經營')
        勞退目標基金的名稱.pop(-4)
        勞退目標基金的名稱.pop(-3)
        print(勞退目標基金的名稱)
        print(有勞退目標基金的頁數_去重複)

        #這邊比較特別，因為有發現表格較為混雜的情況，所以這邊的作法是找到所有目標表格在地頁數，後續用try except找到可以處理的表格
        YF = pd.DataFrame({})
        PDF_file = pr.open(input_path+'/'+PDF_files[2])
        table_name_index = 0
        ps = PDF_file.pages
        for page in 有勞退目標基金的頁數_去重複:
            pg = ps[page]
            #取單頁全部的table
            page_tables = pg.extract_tables()
            for page_table in range(len(page_tables)):
                print(page_table)
                try:
                    table_勞退 = pd.DataFrame(page_tables[page_table])
                    #處理表格內的資料
                    table_勞退.iloc[0] = table_勞退.iloc[0].fillna('',axis=0)
                    table_勞退.iloc[1] = table_勞退.iloc[1].fillna('',axis=0)
                    Date_Start = re.search(r'(\d+.\d+.\d+)~(\d+.\d+.\d+)',table_勞退.iloc[0,4]).group(1)
                    Date_End = re.search(r'(\d+.\d+.\d+)~(\d+.\d+.\d+)',table_勞退.iloc[0,4]).group(2)
                    table_勞退.iloc[0,4] = re.sub(r'(\d+.\d+.\d+)~(\d+.\d+.\d+)','',table_勞退.iloc[0,4])
                    table_勞退_final = table_勞退.iloc[2:,:]
                    table_勞退_final.columns = ['SITE','Mandate Size','AUM','YTD %','Since Inception %','Ranking','Target']
                    table_勞退_final['Target'] = table_勞退_final['Target'].fillna(method = 'ffill')
                    table_勞退_final[['Fund','Mandate Type','From','As of']]=['YF',勞退目標基金的名稱[table_name_index],Date_Start,Date_End]
                    table_勞退_final['From'] = (str(int(re.findall(r'\d+.\d+.\d+',Date_Start)[0][:3])+1911)+str(re.findall(r'\d+.\d+.\d+',Date_Start)[0][3:])).replace('.','/')
                    table_勞退_final['As of'] = (str(int(re.findall(r'\d+.\d+.\d+',Date_End)[0][:3])+1911)+str(re.findall(r'\d+.\d+.\d+',Date_End)[0][3:])).replace('.','/')
                    table_勞退_final = table_勞退_final[table_勞退_final['SITE']!='合計']
                    table_name_index += 1
                    YF = YF.append(table_勞退_final)
                except:
                    print(str(page)+'有問題')
                    table_name_index += 0
                

In [ ]:
def old_new_sys(x):
    if '國內' in x:
        system = 'LRF'
    else:
        system = 'LPF'
    return system
YF['Fund'] = YF['Mandate Type'].apply(lambda x:old_new_sys(x))
Final_DF = pd.concat([YF, LIF, NPIF, PSPF],axis=0)
Final_DF = Final_DF[['Fund','Mandate Type','SITE','Mandate Size','AUM','YTD %','Since Inception %','Ranking','Target','From','As of']]
Final_DF['SITE'] = Final_DF['SITE'].str.replace('\n','')
Final_DF = Final_DF.reset_index(drop=True)
Final_DF['Mandate Type'] = Final_DF['Mandate Type'].str.replace(r'\(\S{4}\)','',regex=True)

In [ ]:
for x in range(len(Final_DF)):
    for y in range(len(GFI)):
        if Final_DF.loc[x,'Fund'] == GFI.loc[y,'Fund'] and Final_DF.loc[x,'Mandate Type'] == GFI.loc[y,'Mandate Type'] and Final_DF.loc[x,'SITE'] == GFI.loc[y,'SITE']:
            #排名不更新
            if Final_DF.loc[x,'Ranking'] == '－' or Final_DF.loc[x,'Ranking'] == '–':
                print('ranking no change')
                要更新的資料 = Final_DF.loc[x,['Mandate Size','AUM','YTD %','Since Inception %','Target','From','As of']]
                GFI.loc[y,['Mandate Size','AUM','YTD %','Since Inception %','Target','From','As of']] = 要更新的資料
            else:
                print('ranking change')
                要更新的資料 = Final_DF.loc[x,['Mandate Size','AUM','YTD %','Since Inception %','Ranking','Target','From','As of']]
                GFI.loc[y,['Mandate Size','AUM','YTD %','Since Inception %','Ranking','Target','From','As of']] = 要更新的資料

In [ ]:
GFI.to_csv(r'D:\My Documents\andyhs\桌面\Andy\政府基金運用資訊-202112\output\政府基金資訊page2_dev.csv',index=False,header=True,encoding='utf-8-sig')